[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](
https://colab.research.google.com/github/FBI223/TOMOGRAFIA_UCZENIE_GLEBOKICH/blob/main/TASK2/task2.ipynb)



# Zadanie obowiazkowe : Odpalenie modelu i wizualizacja dzialania modelu

W tej części student samodzielnie **użyje gotowego, wytrenowanego modelu U-Net** do przeprowadzenia **segmentacji tomografii komputerowej płuc (CT)**.

Celem jest:

1. Wczytanie modelu z pliku `.ckpt`,
2. Wczytanie danych 3D konkretnego pacjenta (`.nii.gz`),
3. Przeprowadzenie predykcji maski nowotworu dla każdej warstwy CT,
4. Stworzenie animacji pokazującej segmentację,

Podpowiedz : Uzyj funkcji


In [3]:
!pip install celluloid
!pip install -U imgaug imageio
!pip install pytorch-lightning torch torchvision torchmetrics nibabel tqdm opencv-python
!pip install onedrivedownloader

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 948.0/948.0 kB 22.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 831.6/831.6 kB 23.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 983.2/983.2 kB 53.7 MB/s eta 0:00:00


In [4]:
from onedrivedownloader import download

url = "https://ujchmura-my.sharepoint.com/:u:/g/personal/marcin_sztukowski_student_uj_edu_pl/EURfIwcEpYVKv88gopAwaeoBbNeeBOqWz3z4IkzUaI5uqg?e=aaQSj9"
download(url, filename="Task06_Lung.tar", unzip=False)
!tar -xvf /content/Task06_Lung.tar -C /content/

100%|██████████| 9.16G/9.16G [02:39<00:00, 57.4MiB/s]


Task06_Lung/._dataset.json
Task06_Lung/dataset.json
Task06_Lung/._imagesTr
Task06_Lung/imagesTr/
Task06_Lung/imagesTr/._lung_053.nii.gz
Task06_Lung/imagesTr/lung_053.nii.gz
Task06_Lung/imagesTr/._lung_022.nii.gz
Task06_Lung/imagesTr/lung_022.nii.gz
Task06_Lung/imagesTr/._lung_041.nii.gz
Task06_Lung/imagesTr/lung_041.nii.gz
Task06_Lung/imagesTr/._lung_069.nii.gz
Task06_Lung/imagesTr/lung_069.nii.gz
Task06_Lung/imagesTr/._lung_014.nii.gz
Task06_Lung/imagesTr/lung_014.nii.gz
Task06_Lung/imagesTr/._lung_006.nii.gz
Task06_Lung/imagesTr/lung_006.nii.gz
Task06_Lung/imagesTr/._lung_065.nii.gz
Task06_Lung/imagesTr/lung_065.nii.gz
Task06_Lung/imagesTr/._lung_018.nii.gz
Task06_Lung/imagesTr/lung_018.nii.gz
Task06_Lung/imagesTr/._lung_096.nii.gz
Task06_Lung/imagesTr/lung_096.nii.gz
Task06_Lung/imagesTr/._lung_084.nii.gz
Task06_Lung/imagesTr/lung_084.nii.gz
Task06_Lung/imagesTr/._lung_086.nii.gz
Task06_Lung/imagesTr/lung_086.nii.gz
Task06_Lung/imagesTr/._lung_043.nii.gz
Task06_Lung/imagesTr/lung_04

In [5]:
from onedrivedownloader import download
url = "https://ujchmura-my.sharepoint.com/:u:/g/personal/marcin_sztukowski_student_uj_edu_pl/EeIxSRDsbaROj51n9jl7TDoBTifXVohnxCVymwhyiGrBSQ?e=IRznML"
download(url, filename="epoch=29-step=53759.ckpt", unzip=False)

100%|██████████| 93.4M/93.4M [00:06<00:00, 15.0MiB/s]


'epoch=29-step=53759.ckpt'

In [6]:
# --- Fix for NumPy 2.0 removal of np.sctypes ---
import numpy as np

if not hasattr(np, "sctypes"):
    np.sctypes = {
        'int': [np.int8, np.int16, np.int32, np.int64],
        'uint': [np.uint8, np.uint16, np.uint32, np.uint64],
        'float': [np.float16, np.float32, np.float64],
        'complex': [np.complex64, np.complex128],
        'others': [bool, object, str, bytes],
    }
# ------------------------------------------------

In [7]:



from pathlib import Path
import torch
import imgaug
import imgaug.augmenters as iaa
from imgaug.augmentables.segmaps import SegmentationMapsOnImage
import matplotlib.pyplot as plt
import pytorch_lightning as pl

class DoubleConv(torch.nn.Module):
    """
    Helper Class which implements the intermediate Convolutions
    """
    def __init__(self, in_channels, out_channels):

        super().__init__()
        self.step = torch.nn.Sequential(torch.nn.Conv2d(in_channels, out_channels, 3, padding=1),
                                        torch.nn.ReLU(),
                                        torch.nn.Conv2d(out_channels, out_channels, 3, padding=1),
                                        torch.nn.ReLU())

    def forward(self, X):
        return self.step(X)


class UNet(torch.nn.Module):
    """
    This class implements a UNet for the Segmentation
    We use 3 down- and 3 UpConvolutions and two Convolutions in each step
    """

    def __init__(self):
        """Sets up the U-Net Structure
        """
        super().__init__()


        ############# DOWN #####################
        self.layer1 = DoubleConv(1, 64)  # MRI -> One channel
        self.layer2 = DoubleConv(64, 128)
        self.layer3 = DoubleConv(128, 256)
        self.layer4 = DoubleConv(256, 512)

        #########################################

        ############## UP #######################
        self.layer5 = DoubleConv(512 + 256, 256)
        self.layer6 = DoubleConv(256+128, 128)
        self.layer7 = DoubleConv(128+64, 64)
        self.layer8 = torch.nn.Conv2d(64, 1, 1)  # Binary label -> One class
        #########################################

        self.maxpool = torch.nn.MaxPool2d(2)

    def forward(self, x):

        ####### DownConv 1#########
        x1 = self.layer1(x)
        x1m = self.maxpool(x1)
        ###########################

        ####### DownConv 2#########
        x2 = self.layer2(x1m)
        x2m = self.maxpool(x2)
        ###########################

        ####### DownConv 3#########
        x3 = self.layer3(x2m)
        x3m = self.maxpool(x3)
        ###########################

        ##### Intermediate Layer ##
        x4 = self.layer4(x3m)
        ###########################

        ####### UpCONV 1#########
        x5 = torch.nn.Upsample(scale_factor=2, mode="bilinear")(x4)  # Upsample with a factor of 2
        #x5 = torch.nn.ConvTranspose2d(512, 512, 2, 2)(x4)
        x5 = torch.cat([x5, x3], dim=1)  # Skip-Connection
        x5 = self.layer5(x5)
        ###########################

        ####### UpCONV 2#########
        x6 = torch.nn.Upsample(scale_factor=2, mode="bilinear")(x5)
        #x6 = torch.nn.ConvTranspose2d(256, 256, 2, 2)(x5)
        x6 = torch.cat([x6, x2], dim=1)  # Skip-Connection
        x6 = self.layer6(x6)
        ###########################

        ####### UpCONV 3#########
        x7 = torch.nn.Upsample(scale_factor=2, mode="bilinear")(x6)
        #x7 = torch.nn.ConvTranspose2d(128, 128, 2, 2)(x6)
        x7 = torch.cat([x7, x1], dim=1)
        x7 = self.layer7(x7)
        ###########################

        ####### Predicted segmentation#########
        ret = self.layer8(x7)
        return ret

class TumorSegmentation(pl.LightningModule):
    def __init__(self):
        super().__init__()

        self.model = UNet()

        self.optimizer = torch.optim.Adam(self.model.parameters(), lr=1e-4)
        self.loss_fn = torch.nn.BCEWithLogitsLoss()

    def forward(self, data):
        pred = self.model(data)
        return pred

    def training_step(self, batch, batch_idx):
        ct, mask = batch
        mask = mask.float()

        pred = self(ct)
        loss = self.loss_fn(pred, mask)

        # Logs
        self.log("Train_Dice", loss)
        if batch_idx % 50 == 0:
            self.log_images(ct.cpu(), pred.cpu(), mask.cpu(), "Train")
        return loss


    def validation_step(self, batch, batch_idx):
        ct, mask = batch
        mask = mask.float()

        pred = self(ct)
        loss = self.loss_fn(pred, mask)

        # Logs
        self.log("Val_Dice", loss)
        if batch_idx % 50 == 0:
            self.log_images(ct.cpu(), pred.cpu(), mask.cpu(), "Val")

        return loss


    def log_images(self, ct, pred, mask, name):

        results = []

        pred = pred > 0.5 # As we use the sigomid activation function, we threshold at 0.5


        fig, axis = plt.subplots(1, 2)
        axis[0].imshow(ct[0][0], cmap="bone")
        mask_ = np.ma.masked_where(mask[0][0]==0, mask[0][0])
        axis[0].imshow(mask_, alpha=0.6)
        axis[0].set_title("Ground Truth")

        axis[1].imshow(ct[0][0], cmap="bone")
        mask_ = np.ma.masked_where(pred[0][0]==0, pred[0][0])
        axis[1].imshow(mask_, alpha=0.6, cmap="autumn")
        axis[1].set_title("Pred")

        self.logger.experiment.add_figure(f"{name} Prediction vs Label", fig, self.global_step)



    def configure_optimizers(self):
        #Caution! You always need to return a list here (just pack your optimizer into one :))
        return [self.optimizer]



In [8]:
import torch
import cv2
from tqdm import tqdm
import matplotlib.pyplot as plt
from celluloid import Camera
from IPython.display import HTML, Video
import numpy as np
import warnings
from matplotlib.animation import FuncAnimation
import os


def load_trained_model(checkpoint_path, model_class = TumorSegmentation, device=None):
    """
    Wczytuje wytrenowany model PyTorch Lightning z pliku .ckpt

    Args:
        checkpoint_path: ścieżka do pliku checkpoint (.ckpt)
        model_class: klasa modelu (np. TumorSegmentation)
        device: 'cuda', 'cpu' lub None (automatycznie wykrywane)

    Returns:
        model: załadowany i gotowy do ewaluacji model
    """

    # automatyczne wykrycie urządzenia
    if device is None:
        device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

    # wczytanie modelu z checkpointu
    print(f"🔄 Loading model from: {checkpoint_path}")
    model = model_class.load_from_checkpoint(checkpoint_path)

    # przeniesienie na GPU/CPU
    model = model.to(device)
    model.eval()

    print(f"✅ Model loaded successfully on {device}")
    return model





def create_segmentation_video(scan, segmentation, save_path="/content/ct_tumor_animation.mp4",
                              step=2, fps=10):
    """
    Tworzy wideo MP4 z segmentacji CT bez użycia matplotlib.animation (działa w Colabie bez błędów).
    """
    os.makedirs("/content/tmp_frames", exist_ok=True)
    tmp_dir = "/content/tmp_frames"

    # --- zapis pojedynczych klatek ---
    for idx, i in enumerate(tqdm(range(0, len(scan), step), desc="Rendering frames")):
        plt.figure(figsize=(6, 6))
        plt.imshow(scan[i], cmap="bone")
        mask = np.ma.masked_where(segmentation[i] == 0, segmentation[i])
        plt.imshow(mask, alpha=0.5, cmap="autumn")
        plt.axis("off")
        frame_path = f"{tmp_dir}/frame_{idx:04d}.png"
        plt.savefig(frame_path, bbox_inches="tight", pad_inches=0)
        plt.close()

    # --- składanie filmu przez ffmpeg ---
    print("💾 Rendering wideo przez ffmpeg...")
    cmd = f"ffmpeg -y -framerate {fps} -i {tmp_dir}/frame_%04d.png -c:v libx264 -pix_fmt yuv420p {save_path}"
    os.system(cmd)

    # --- czyszczenie ---
    for f in os.listdir(tmp_dir):
        os.remove(os.path.join(tmp_dir, f))
    os.rmdir(tmp_dir)

    print(f"✅ Zapisano: {save_path}")




def predict_single_patient(model, ct_volume, device="cuda", threshold=0.5):
    """
    Segmentacja jednego pacjenta (CT volume) przy użyciu modelu (np. U-Net).

    Args:
        model: wytrenowany model PyTorch (np. U-Net)
        ct_volume: numpy array 3D (H, W, N) — pełny tomogram pacjenta
        device: 'cuda' lub 'cpu'
        threshold: próg binarny dla maski (np. 0.5)

    Returns:
        scan: lista 2D obrazów (oryginalnych slice'ów)
        segmentation: lista 2D masek binarnych
        preds: lista masek z wartościami sigmoid (float)
    """

    model.eval()
    scan = []
    segmentation = []
    preds = []

    for i in tqdm(range(ct_volume.shape[-1])):
        # --- przygotowanie slice’a ---
        slice_ = cv2.resize(ct_volume[:, :, i], (256, 256))
        scan.append(slice_)

        input_t = torch.tensor(slice_).unsqueeze(0).unsqueeze(0).float().to(device)

        # --- predykcja modelu ---
        with torch.no_grad():
            pred = torch.sigmoid(model(input_t))[0, 0].cpu().numpy()

        preds.append(pred)
        segmentation.append((pred > threshold).astype(np.uint8))

    scan = np.array(scan)
    segmentation = np.array(segmentation)
    preds = np.array(preds)

    return scan, segmentation, preds




In [9]:



# ======================================================
# 👨‍🏫 CZĘŚĆ NAUCZYCIELA
# ======================================================

import torch
import cv2
import numpy as np
import matplotlib.pyplot as plt
from celluloid import Camera
from IPython.display import Video
from tqdm import tqdm
import nibabel as nib


THRESHOLD = 0.5
MAX_VAL = 3071
CROP_SIZE = 30
#stale do stworzenia wideo
STEPS = 2
FPS=10


DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
checkpoint = "/content/epoch=29-step=53759.ckpt"
model = load_trained_model(checkpoint_path=checkpoint, model_class=TumorSegmentation)
subject = Path("/content/Task06_Lung/imagesTs/lung_035.nii.gz")
ct = nib.load(subject).get_fdata() / MAX_VAL  # standardize
ct = ct[:,:,CROP_SIZE:]  # crop
scan, segmentation, preds = predict_single_patient(model, ct, device=DEVICE)
create_segmentation_video(scan, segmentation, "/content/ct_tumor_animation_35.mp4", step=STEPS, fps=FPS)
Video("/content/ct_tumor_animation_35.mp4", embed=True)




🔄 Loading model from: /content/epoch=29-step=53759.ckpt


/usr/local/lib/python3.12/dist-packages/pytorch_lightning/utilities/migration/migration.py:208: You have multiple `ModelCheckpoint` callback states in this checkpoint, but we found state keys that would end up colliding with each other after an upgrade, which means we can't differentiate which of your checkpoint callbacks needs which states. At least one of your `ModelCheckpoint` callbacks will not be able to reload the state.
INFO:pytorch_lightning.utilities.migration.utils:Lightning automatically upgraded your loaded checkpoint from v1.3.4 to v2.5.6. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint epoch=29-step=53759.ckpt`


✅ Model loaded successfully on cuda:0


Rendering frames: 100%|██████████| 146/146 [00:34<00:00,  4.20it/s]


💾 Rendering wideo przez ffmpeg...
✅ Zapisano: /content/ct_tumor_animation_35.mp4


In [10]:



# ======================================================
# 👩‍🎓 CZĘŚĆ STUDENTA
# ======================================================



import torch
import cv2
import numpy as np
import matplotlib.pyplot as plt
from celluloid import Camera
from IPython.display import Video
from tqdm import tqdm

import nibabel as nib



THRESHOLD = 0.5
MAX_VAL = 3071
CROP_SIZE = 30
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

#stale do stworzenia wideo
STEPS = 2
FPS=10


# wczytaj pobrany model / checkpoint
checkpoint = "/content/POBRANY_MODEL..." # TODO
model = load_trained_model( ) #TODO
subject = Path("/content/Task06_Lung/imagesTs/DANE_PACJENTA.nii.gz") #TODO




# Uzyj nib.load oraz get_fdata aby wczytac dane dowolnego pacjenta
# pamietaj aby podzielic przez MAX_VAL wczytane dane aby ustandaryzowac dane
# pamietaj aby zrobic CROP wczytanych danych

subject = Path("/content/Task06_Lung/imagesTs/WYBIERZ_ZDJECIE") #TODO
ct = nib.load  #TODO load
ct = ct[...]  # TODO crop
scan, segmentation, preds =  ...  #TODO uzyj funkcji wyzej

#uzyj gotowej funkcji do stworzenia video z klatek   #TODO
Video("/content/VIDEO.mp4", embed=True) #TODO




TypeError: load_trained_model() missing 1 required positional argument: 'checkpoint_path'